In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline
import time
import os
import csv
import sklearn
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.utils import resample
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
# from auto_ml import Predictor
# from auto_ml.utils import get_boston_dataset
# from auto_ml.utils_models import load_ml_model
from numpy import loadtxt
from xgboost import XGBClassifier
import xgboost as xgb

In [ ]:
bank_train = pd.read_csv("bankruptcy_Train.csv")
bank_train.shape

In [ ]:
bank_test = pd.read_csv('bankruptcy_Test_X.csv')

In [ ]:
X = bank_train.iloc[:,0:64]  #independent columns
y = bank_train.iloc[:,-1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
K = 5
kf = KFold(n_splits = K, random_state = 3228, shuffle = True)

smote = SMOTE(ratio='minority')
X_train, y_train = smote.fit_sample(X_train, y_train)

X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)
y_train.rename(columns={0:"class"}, inplace=True)

In [ ]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import model_selection, metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

target = 'class'
IDcol = 'ID'
train = X_train.join(y_train)

In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['class'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['class'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['class'], dtrain_predprob))

In [ ]:
train.columns = ['Attr1', 'Attr2', 'Attr3', 'Attr4', 'Attr5', 'Attr6', 
                 'Attr7', 'Attr8', 'Attr9', 'Attr10', 'Attr11', 'Attr12', 'Attr13', 
                 'Attr14', 'Attr15', 'Attr16', 'Attr17', 'Attr18', 'Attr19', 'Attr20', 
                 'Attr21', 'Attr22', 'Attr23', 'Attr24', 'Attr25', 'Attr26', 'Attr27', 
                 'Attr28', 'Attr29', 'Attr30', 'Attr31', 'Attr32', 'Attr33', 'Attr34', 
                 'Attr35', 'Attr36', 'Attr37', 'Attr38', 'Attr39', 'Attr40', 'Attr41', 
                 'Attr42', 'Attr43', 'Attr44', 'Attr45', 'Attr46', 'Attr47', 'Attr48', 
                 'Attr49', 'Attr50', 'Attr51', 'Attr52', 'Attr53', 'Attr54', 'Attr55', 
                 'Attr56', 'Attr57', 'Attr58', 'Attr59', 'Attr60', 'Attr61', 'Attr62', 
                 'Attr63', 'Attr64','class']

In [ ]:
predictors = [x for x in train.columns if x not in [target, IDcol]]

In [ ]:
xgb2 = XGBClassifier(
     learning_rate =0.07,
     n_estimators=4000,
     max_depth=6,
     min_child_weight=1,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.9,
     objective= 'binary:logistic',
     nthread=4,
     scale_pos_weight=1,
     seed=27)
print(modelfit(xgb2, train, predictors))

predictions = xgb2.predict(X_test)
#print(i)
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

print("Classification Report")
print(classification_report(y_test, predictions))

prob_y_5 = xgb2.predict_proba(X_test)
prob_y_5 = [p[1] for p in prob_y_5]
print( roc_auc_score(y_test, prob_y_5))

In [ ]:
prob_y_5 = gsearch1.predict_proba(bank_test)
dataframe=pd.DataFrame(prob_y_5) 
dataframe.to_csv('output.csv')